# Using `PSAIdentifier` to extract PSA data

Here we will use the convenience class `PSAIdentifier` to extract Hausdorff pair analysis data generated by PSA.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Set up input data for `PSA` using `MDAnalysis`

In [2]:
from MDAnalysis import Universe
from MDAnalysis.analysis.psa import PSA
from psa_identifier import PSAIdentifier

Initialize lists for the methods on which to perform PSA. PSA will be performed for four different simulations methods with three runs for each: **DIMS**, **FRODA**, **rTMD-F**, and **rTMD-S**. Also initialize a `PSAIdentifier` object to keep track of the data corresponding to comparisons between pairs of simulations.

In [3]:
method_names = ['DIMS','FRODA','rTMD-F','rTMD-S']
labels = [] # Heat map labels (not plotted in this example)
simulations = [] # List of simulation topology/trajectory filename pairs
universes = [] # List of MDAnalysis Universes representing simulations

For each method, get the topology and each of three total trajectories (per method). Each simulation is represented as a `(topology, trajectory)` pair of file names, which is appended to a master list of simulations.

In [4]:
for method in method_names:
    # Note: DIMS uses the PSF topology format
    topname = 'top.psf' if 'DIMS' in method or 'TMD' in method else 'top.pdb'
    pathname = 'fitted_psa.dcd'
    method_dir = 'methods/{}'.format(method)
    if method is not 'LinInt':
        for run in xrange(1, 4): # 3 runs per method
            run_dir = '{}/{:03n}'.format(method_dir, run)
            topology = '{}/{}'.format(method_dir, topname)
            trajectory = '{}/{}'.format(run_dir, pathname)
            labels.append(method + '(' + str(run) + ')')
            simulations.append((topology, trajectory))
    else: # only one LinInt trajectory
        topology = '{}/{}'.format(method_dir, topname)
        trajectory = '{}/{}'.format(method_dir, pathname)
        labels.append(method)
        simulations.append((topology, trajectory))

Generate a list of universes from the list of simulations.

In [5]:
for sim in simulations:
    universes.append(Universe(*sim))

## Perform a path similarity analysis

Initialize a PSA comparison from the universe list using a C$_\alpha$ trajectory representation.

In [6]:
psa_hpa = PSA(universes, path_select='name CA', labels=labels)

Generate PSA `Path`s from the trajectories

In [7]:
psa_hpa.generate_paths()

Perform a Hausdorff pairs analysis on all of the `Path`s

In [9]:
psa_hpa.run_hausdorff_pairs_analysis(hausdorff_pairs=True)

## Extract specific data from PSA

In [10]:
psa_id = PSAIdentifier()
for name in method_names:
    psa_id.add_sim(name, [1,2,3])

Get the PSA ID for the second DIMS simulation (DIMS 2) and third rTMD-F simulation (rTMD-F 3).

In [11]:
ID = psa_id.get_psa_id('DIMS 2', 'rTMD-F 3')

### Use the PSA ID to locate the Hausdorff analysis data for the DIMS 2/rTMD-F 3 comparison:

Get the indices of the frames for the DIMS 2 and rTMD-F 3 paths corresponding to the Hausdorff pair.

In [12]:
psa_hpa.HP[ID]['frames']

(48, 97)

Get the rmsd separating the Hausdorff pair (this is the Hausdorff distance!)

In [13]:
psa_hpa.HP[ID]['distance']

1.8656396

In [14]:
df = psa_id.data

In [15]:
df

Sim ID
Name   Run ID        
DIMS   1            0
       2            1
       3            2
FRODA  1            3
       2            4
       3            5
rTMD-F 1            6
       2            7
       3            8
rTMD-S 1            9
       2           10
       3           11

In [24]:
df.loc[('DIMS',[1,2,3]), 'Sim ID']

Name  Run ID
DIMS  1         0
      2         1
      3         2
Name: Sim ID, dtype: int64